In [18]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm_notebook
import pandas as pd

In [19]:
# 1. 종목분석 페이지 url을 만든다.
def get_page(pagenum):
    page_base = 'https://finance.naver.com/research/company_list.nhn?&page={}'
    page = page_base.format(pagenum)
    return page

In [20]:
pagenum = 1
page = 'https://finance.naver.com/research/company_list.nhn?&page={}'.format(pagenum)
page

'https://finance.naver.com/research/company_list.nhn?&page=1'

In [21]:
get_page(1)

'https://finance.naver.com/research/company_list.nhn?&page=1'

In [22]:
# 2. 한 페이지당 종목분석 리포트 링크를 가져온다.
def get_links(page):

    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36' }
    html = requests.get(page, headers = headers).text
    time.sleep(1)
    soup = BeautifulSoup(html, 'html.parser')
    
    links_all = soup.select('div.box_type_m tr > td > a')
    links = []
    for lll in links_all:
        if 'company_read.nhn?' in lll['href']:
            link = 'https://finance.naver.com/research/' + lll['href']
            links.append(link)
    return links

In [23]:
page = get_page(1)
get_links(page)

['https://finance.naver.com/research/company_read.nhn?nid=46892&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46891&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46890&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46889&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46888&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46887&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46886&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46885&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46884&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46883&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46882&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46881&page=1',
 'https://finance.naver.com/research/company_read.nhn?nid=46880&page=1',
 'https://finance.naver.com/research/company_read.n

In [24]:
# 3. 링크 당 정보를 가져온다. (제목, 내용, 일시, 종목, 검토회사...)
def get_report(link):

    headers = {'Referer' : 'https://finance.naver.com/research/invest_list.nhn', 'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36' }
    html = requests.get(link, headers = headers).text
    time.sleep(1)
    soup = BeautifulSoup(html, 'html.parser')

    # print(soup.select('em'))

    stock = soup.select('em')[0].text
    target = soup.select('em > strong')[0].text
    reco = soup.select('em.coment')[0].text

    title_del = soup.select('th.view_sbj > p.source')[0].text
    title = soup.select('th.view_sbj')[0].text.replace(title_del, '').replace(stock, '').strip()
    report = soup.select('td.view_cnt > div')[0].text.replace('\n', '').strip()

    company = title_del.split('|')[0]
    date = title_del.split('|')[1]
    pdf = soup.select('a.con_link')[0]['href']
    
    result = [stock, target, reco, title, report, company, date, pdf]
    
    return result

# 이렇게 하면 안됨 ㅠㅠ

In [25]:
link = 'https://finance.naver.com/research/company_read.nhn?nid=46892&page=1'
headers = {'Referer' : 'https://finance.naver.com/research/invest_list.nhn', 'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36' }
html = requests.get(link, headers = headers).text
time.sleep(1)
soup = BeautifulSoup(html, 'html.parser')

#print(html)
print(soup.select('em'))

[<em>현대백화점</em>, <em class="money"><strong>125000</strong>원</em>, <em class="coment">매수</em>, <em>Copyright ©</em>]


# 이렇게 하면 됨..

In [26]:
link = 'https://finance.naver.com/research/company_read.nhn?nid=46892&page=1'
get_report(link)

['현대백화점',
 '125000',
 '매수',
 '2019년, 기대 해볼만하다',
 '백화점, 2019년 문제 없다현대백화점 3분기 총매출액 1조 4,045억원(4% YoY), 영업이익 771억원 (11% YoY)을 예상한다.',
 '미래에셋대우',
 '2018.10.04',
 'https://ssl.pstatic.net/imgstock/upload/research/company/1538621713176.pdf']

# get_report로 그냥 링크를 넣으면, em 태그 값이 나오는데, 함수에서 통으로 넣으면 빈 리스트가 나옴 --> 체크 필요

In [49]:
# 4. 데이터 수집하기
def get_stock_report(totalpage, fname):

    #-*- coding: utf-8 -*-

    links = []
    
    for i in tqdm_notebook(range(1, totalpage), desc = '점검중'):
        page = get_page(i)
        links += get_links(page)
        
#    print(page, links)
    print('{}개의 투자전략 리포트를 찾았습니다.' .format(len(links)))
    print('리포트 수집 진행합니다. \n')
    
    results = []
    
    for link in tqdm_notebook(links, desc = '리포트'):
        # print(link)
        result = get_report(link)
        results.append(result)
        #print(result)
    
    #print(results)
    datas = pd.DataFrame(results)
    datas.columns = ['종목명', '목표가', '의견', '제목', '내용', '회사', '일자', '보고서']
#    datas.to_csv(fname, encoding = 'EUC-KR')
# 인코딩을 utf-8나 cp949로 해도 모두 깨짐
# http://www.hellogohn.com/post_one52에서 말한대로, 엑셀파일로 export시키니까 됨

    datas.to_excel(fname, sheet_name = 'sheet1')
    print('{}개의 투자전략 리포트 수집 완료.'.format(len(results)))

In [51]:
totalpage = 2
fname = '투자전략리포트.xlsx'

In [52]:
get_stock_report(totalpage, fname)

30개의 투자전략 리포트를 찾았습니다.
리포트 수집 진행합니다. 



30개의 투자전략 리포트 수집 완료.
